Aqui temos a instalação das bibliotecas necessárias

In [1]:
!python -m pip install crewai
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install crewai_tools

Importando as classes Agent, Task e Crew, as quais são respectivamente responsáveis por nos permitir criar nossos agentes, criar as tarefas que nossos agentes vão desempenhar e por fim realizar a orquestração. LLM foi inserido pois não estou conseguindo usar meus créditos da OpenAI.

In [3]:
from crewai import Agent, Task, Crew, LLM

LLM que vai se conectar aos nosso agente e credenciais, pelo fato de não conseguir usar a da OpenAI vou usar minhas credenciais do gemini da google e devido a isso vou inserir, o parâmetro llm em todos os agentes para que eles usem o modelo gemini. Além disso temos a chave de api do server e do firecrawl as quais serão usadas pelas respectivas apis.

In [4]:
import os
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = LLM(
              model='gemini/gemini-1.5-flash',
              api_key=os.environ["GEMINI_API_KEY"]
            )
os.environ["SERPER_API_KEY"] = userdata.get('SERPER-EI')
firecrawl_api_key= userdata.get('fire')

Abaixo temos nosso primeiro agente o agente_busca_hoteis, o qual conta com a classe Agent definida anteriormente, a qual conta com os parâmetros role, goal e backstory, os quais são responsáveis respectivamente por dizer qual a função que o agente ocupa em nossas tarefas, o seu objetivo geral é encontar os melhores hotéis disponíveis que atendam aaos critérios do cliente e uma história de fundo que gera contexto para o agente. Além disso temos os parâmetros allow_delegation e verbose, os quais são responsáveis respectivamente por permitir que o agente atribua tarefa para outro, que no caso do exemplo é possível pois está como True a verbose que mostra como são os pensamentos do agente e temos também memory que guarda as memórias das interações do agente.

In [5]:
agente_busca_hoteis = Agent(
    role="Especialista em Busca de Hotéis",
    goal="Encontrar os melhores hotéis disponíveis que atendam aos critérios do cliente",
    backstory=(
        "Você trabalha em uma agência de viagens e é especialista em encontrar os melhores hotéis "
        "para os clientes com base em suas preferências. "
        "Seu trabalho é pesquisar hotéis {quantidade} na região {regiao}, garantindo que estejam dentro da "
        "faixa de preço de {faixa_preco} e disponíveis entre {data_inicio} e {data_fim}. "
        "Você deve utilizar fontes online e plataformas de reservas para coletar informações precisas e atualizadas."
    ),
    allow_delegation=False,
    verbose=True,
    memory=True,
    llm=llm
)

Abaixo temos nosso segundo agente o agente_qualidade_hoteis, o qual conta com a classe Agent definida anteriormente, a qual conta com os parâmetros role, goal e backstory, os quais são responsáveis respectivamente por dizer qual a função que o agente ocupa em nossas tarefas, o seu objetivo geral é garantir que os hotéis recomendados atendam aos requisitos do cliente e aos padrões de qualidade e uma história de fundo que gera contexto para o agente. Além disso temos os parâmetros allow_delegation e verbose, os quais são responsáveis respectivamente por permitir que o agente atribua tarefa para outro, que no caso do exemplo é possível pois está como True a verbose que mostra como são os pensamentos do agente e temos também memory que guarda as memórias das interações do agente.

In [6]:
agente_qualidade_hoteis = Agent(
    role="Especialista em Garantia de Qualidade de Busca de Hotéis",
    goal="Garantir que os hotéis recomendados atendam aos requisitos do cliente e aos padrões de qualidade",
    backstory=(
        "Você trabalha em uma agência de viagens e sua função é revisar os resultados da busca de {quantidade} hotéis "
        "para garantir que atendam às expectativas do cliente. "
        "Você deve verificar se os hotéis encontrados na região {regiao} estão dentro da {faixa_preco} "
        "e disponíveis de {data_inicio} a {data_fim}. "
        "Verifique se todas as informações fornecidas são precisas, relevantes e corretamente referenciadas. "
        "Sua revisão garante que o cliente receba as melhores opções possíveis."
    ),
    verbose=True,
    memory=True,
    llm=llm
)

Abaixo instalei o boto 3, pois estava tendo um problema de dependencias ao executar o próximo trecho

In [7]:
!pip install boto3

Abaixo são importadas as classes SerperDevTool que usa a API do Serper para buscar resultados do google, a classe ScrapeWebsiteTool que faz scraping de páginas da web coletando extos, links e outras informações e a classe WebsiteSearchTool que realiza pesquisas dentro de um site encontrando termos específicos

In [8]:
from crewai_tools import SerperDevTool, FirecrawlScrapeWebsiteTool

In [9]:
!pip install firecrawl-py

Abaixo é definido o site que será realizado o scraping, o qual foi inserido no parâmetro website_url para busca_hoteis, há também a pesquisa de disponibilidade e a avalização de hoteis usando o SerperDevTool

In [10]:
busca_hoteis_tool = FirecrawlScrapeWebsiteTool(api_key=firecrawl_api_key)
pesquis_e_avaliacoes_hoteis_tool = SerperDevTool()

Abaixo temos a a variável tarefa_busca_hoteis, a qual é uma das tasks que nossos agentes podem realizar, dessa forma temos como parâmetros, a descrição da task, onde são inseridos uma série de passos e recomendações que devem ser seguidas na realização da task, o expect_output, que se trata de como deve ser a resposta do agente a tarefa, a qual deve fornecer uma lista detalhada de hoteis que atendem aos criteirios definidos pelo ciente como data de inicio/fim, faixa de preço e região, nessa lsia deve conter o nome do hotel as informações já solicitadas, o link de reserva, avaliação de clientes, referências e comodidades incluidas. Além disso a tarefa deve ser feita usando as  tools busca_hoteis_tool, pesquisa_e_avaliacoes_hoteis_tool pelo agente_busca_hoteis.

In [11]:
tarefa_busca_hoteis = Task(
        description=(
        "Pesquise hotéis na região de **{regiao}** dentro da faixa de preço de **{faixa_preco}**, "
        "para o período de **{data_inicio} a {data_fim}**.\n\n"
        "Utilize fontes confiáveis como **Google Hotels, Kayak e Hoteis.com**, priorizando hotéis bem avaliados e com boas comodidades. "
        "Não utilize o Booking.com"
        "Certifique-se de que as opções encontradas estão disponíveis para as datas especificadas e dentro do orçamento definido.\n\n"
        "Apresente os resultados em uma **tabela estruturada**, contendo:\n"
        "- **Nome do Hotel**\n"
        "- **Localização**\n"
        "- **Preço por Noite**\n"
        "- **Avaliação Média**\n"
        "- **Comodidades Principais**\n"
        "- **Disponibilidade**"
    ),
    expected_output=(
        "Uma lista detalhada com os melhores hotéis que atendem às preferências do cliente, "
        "incluindo nome, localização, preço por noite, comodidades e avaliações relevantes."
    ),
    tools=[busca_hoteis_tool, pesquis_e_avaliacoes_hoteis_tool],
    agent=agente_busca_hoteis
)

Abaixo temos a a variável tarefa_revisao_qualidade_hoteis, a qual é uma das tasks que nossos agentes podem realizar, dessa forma temos como parâmetros, a descrição da task, onde são inseridos uma série de passos e recomendações que devem ser seguidas na realização da task, o expect_output, que se trata de como deve ser a resposta do agente a tarefa, a qual deve fornecer uma lista refinada da resposta do agente anterior, certificando de que as infomações são precisas, a disponibilidade dos hotéis e a relação custo benefício e o parâmetro agent, que atribui o agente responsável pela tarefa, sendo no caso do exemplo abaixo o agente_qualidade_hoteis criado anteriormente.

In [12]:
tarefa_revisao_qualidade_hoteis = Task(
    description=(
        "Revise cuidadosamente a lista de hotéis selecionados para um cliente que deseja se hospedar na região de **{regiao}**, "
        "dentro da faixa de preço de **{faixa_preco}**, para uma estadia de **{data_inicio}** a **{data_fim}**.\n\n"
        "Verifique se os hotéis atendem aos critérios definidos, garantindo que os preços, disponibilidade e avaliações "
        "sejam precisos e confiáveis. Certifique-se de que os hotéis oferecem um bom custo-benefício "
        "com base em localização, comodidades e avaliações de hóspedes.\n\n"
        "Apresente os resultados em uma **tabela estruturada**, contendo as seguintes informações: \n"
        "- **Nome do Hotel**\n"
        "- **Localização**\n"
        "- **Preço por Noite**\n"
        "- **Avaliação Média**\n"
        "- **Comodidades Principais**\n"
        "- **Disponibilidade**\n"
        "Não precisa colocar observações"
    ),
    expected_output=(
        "Uma tabela com as seguintes colunas:"
        "Nome do Hotel,"
        "Localização,"
        "Preço por Noite,"
        "Avaliação Média,"
        "Comodidades Principais e "
        "Disponibilidade."
        "Sem observações na tabela"
    ),
    agent=agente_qualidade_hoteis
)


Abaixo temos a variável crew, a qual recebe a classe Crew, onde temos os seguintes parâmetros agents, tasks e verbose, os quais respectivamente se tratam de uma lista de agentes os quais vão executar as tarefas, a lista de tarefas a serem executadas por nossos agentes e os pensamentos de nossos agentes ao executar as tarefas. Além vale destacar que nesse caso as tarefas são executadas sequencialmente, isto é inquiry_resolution -> quality_assurance_review. Aqui diferente do tutorial a verbose teve que ser ajustado para true, devido as versões mais recentes da biblioteca.

In [13]:
crew = Crew(
    agents=[agente_busca_hoteis, agente_qualidade_hoteis],
    tasks=[tarefa_busca_hoteis, tarefa_revisao_qualidade_hoteis],
    verbose=True
)

Abaixo temos a variável results a qual recebe o metódo kickoff da variável crew, onde temos os inputs que se tratam de valores interpolados que podemos substituir nos agentes, ou seja, para todas as vezes que apareceu a palavra customer tanto na definição dos agentes ou das tasks podemos substituir regiao por Rio de Janeiro, Brasil, o mesmo vale para faixa_preco que é substituida por 300 - 700 por noite e as datas de inicio e fim definidas abaixo, e ao executar podemos observar a execução de cada tarefa uma por uma devido ao fato de verbose estar ativa, como declarado anteriormente.

In [14]:
inputs = {
    "regiao": "Rio de Janeiro, Brasil",
    "faixa_preco": "R$ 300 - R$ 3000 por noite",
    "data_inicio": "2025-07-10",
    "data_fim": "2025-07-15",
    "quantidade": 3
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2f5fa482-7513-43a7-afc1-758704c9f2eb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: ef0d2093-99e6-4435-ac0d-0a7c996489b6
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: ef0d2093-99e6-4435-ac0d-0a7c996489b6
       Status: Executing Task...
    └── 🤖 Agent: Especialista em Busca de Hotéis
            Status: In Progress

# Agent: Especialista em Busca de Hotéis
## Task: Pesquise hotéis na região de **Rio de Janeiro, Brasil** dentro da faixa de preço de **R$ 300 - R$ 3000 por noite**, para o período de **2025-07-10 a 2025-07-15**.

Utilize fontes confiáveis como **Google Hotels, Kayak e Hoteis.com**, priorizando hotéis bem avaliados e com boas comodidades. Não utilize o Booking.comCertifique-se de que as opções encontradas estão disponíveis para as datas especificadas e dentro do orçamento definido.

Apresente os resultados em uma **tabela estruturada**, contendo:
- **Nome do Hotel**
- **Localização**
- **Preço por Noite**
- **Avaliação Média**
- **Comodidades Principais**
- **Disponibilidade**


🚀 Crew: crew
└── 📋 Task: ef0d2093-99e6-4435-ac0d-0a7c996489b6
       Status: Executing Task...
    └── 🤖 Agent: Especialista em Busca de Hotéis
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Especialista em Garantia de Qualidade de Busca de Hotéis
## Final Answer: 
| Nome do Hotel             | Localização        | Preço por Noite (estimativa) | Avaliação Média (estimativa) | Comodidades Principais                                    | Disponibilidade (estimativa) |
|--------------------------|--------------------|-----------------------------|-----------------------------|---------------------------------------------------------|-----------------------------|
| Windsor Martinique       | Copacabana          | R$ 800 - R$ 1.500            | 4.0                          | Piscina, restaurante, localização central                 | Consultar site               |
| Miramar Hotel by Windsor | Copacabana          | R$ 700 - R$ 1.200            | 4.0                          | Piscina, restaurante, vista para o mar                 | Consultar site               |
| Ipanema Inn Hotel         | Ipanema            | R$ 500 - R$ 1.000            | 3.5                      

🚀 Crew: crew
├── 📋 Task: ef0d2093-99e6-4435-ac0d-0a7c996489b6
│      Assigned to: Especialista em Busca de Hotéis
│      Status: ✅ Completed
│   ├── 🤖 Agent: Especialista em Busca de Hotéis
│   │       Status: In Progress
│   │   ├── 🔧 Used Search the internet with Serper (2)
│   │   └── 🔧 Used Firecrawl web scrape tool (1)
│   ├── 🤖 Agent: Especialista em Busca de Hotéis
│   │       Status: In Progress
│   │   ├── 🔧 Used Search the internet with Serper (4)
│   │   └── 🔧 Used Firecrawl web scrape tool (2)
│   └── 🤖 Agent: Especialista em Busca de Hotéis
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (6)
└── 📋 Task: 5db97e78-1dd7-401d-b632-6c69d940d083
       Status: Executing Task...
    └── 🤖 Agent: Especialista em Garantia de Qualidade de Busca de Hotéis
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: ef0d2093-99e6-4435-ac0d-0a7c996489b6
│      Assigned to: Especialista em Busca de Hotéis
│      Status: ✅ Completed
│   ├── 🤖 Agent: Especialista em Busca de Hotéis
│   │       Status: In Progress
│   │   ├── 🔧 Used Search the internet with Serper (2)
│   │   └── 🔧 Used Firecrawl web scrape tool (1)
│   ├── 🤖 Agent: Especialista em Busca de Hotéis
│   │       Status: In Progress
│   │   ├── 🔧 Used Search the internet with Serper (4)
│   │   └── 🔧 Used Firecrawl web scrape tool (2)
│   └── 🤖 Agent: Especialista em Busca de Hotéis
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (6)
└── 📋 Task: 5db97e78-1dd7-401d-b632-6c69d940d083
       Assigned to: Especialista em Garantia de Qualidade de Busca de Hotéis
       Status: ✅ Completed
    └── 🤖 Agent: Especialista em Garantia de Qualidade de Busca de Hotéis
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5db97e78-1dd7-401d-b632-6c69d940d083                                                                     │
│  Agent: Especialista em Garantia de Qualidade de Busca de Hotéis                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2f5fa482-7513-43a7-afc1-758704c9f2eb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Abaixo temos a resposta final produzida pelos agentes

In [15]:
from IPython.display import Markdown
Markdown(result.raw)

| Nome do Hotel             | Localização        | Preço por Noite (estimativa) | Avaliação Média (estimativa) | Comodidades Principais                                    | Disponibilidade (estimativa) |
|--------------------------|--------------------|-----------------------------|-----------------------------|---------------------------------------------------------|-----------------------------|
| Windsor Martinique       | Copacabana          | R$ 800 - R$ 1.500            | 4.0                          | Piscina, restaurante, localização central                 | Consultar site               |
| Miramar Hotel by Windsor | Copacabana          | R$ 700 - R$ 1.200            | 4.0                          | Piscina, restaurante, vista para o mar                 | Consultar site               |
| Ipanema Inn Hotel         | Ipanema            | R$ 500 - R$ 1.000            | 3.5                          | Localização em Ipanema, perto da praia                    | Consultar site               |
| Hotel Arpoador            | Ipanema            | R$ 600 - R$ 1.200            | 3.8                          | Localização próxima a praia de Ipanema e Arpoador           | Consultar site               |
| Hotel Santa Teresa        | Santa Teresa        | R$ 800 - R$ 1.800            | 4.2                          | Charme, vista panorâmica, localização no bairro boêmio     | Consultar site               |


**Observação:** Os hotéis Copacabana Palace, Fairmont Rio de Janeiro e Belmond Hotel Copacabana foram excluídos da lista pois seus preços estão acima do limite superior (R$ 3000) definido pelo cliente, mesmo considerando a faixa de preço estimada.  Os preços e a disponibilidade são estimativas e podem variar dependendo da data e da disponibilidade do hotel. É altamente recomendável verificar diretamente nos sites dos hotéis ou plataformas de reserva (Booking.com, Expedia, Decolar.com) para obter informações precisas e reservar.